In [42]:
import zipfile
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge
import numpy as np
from sklearn import linear_model, metrics, model_selection
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import statsmodels.api as sm


In [2]:
zip = zipfile.ZipFile("data/houses.zip")
zip.extractall("data")
zip.close()

In [3]:
houses = pd.read_csv("data/houses.csv")

In [4]:
houses.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


In [5]:
houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2919 non-null   int64  
 1   MSSubClass     2919 non-null   int64  
 2   MSZoning       2915 non-null   object 
 3   LotFrontage    2433 non-null   float64
 4   LotArea        2919 non-null   int64  
 5   Street         2919 non-null   object 
 6   Alley          198 non-null    object 
 7   LotShape       2919 non-null   object 
 8   LandContour    2919 non-null   object 
 9   Utilities      2917 non-null   object 
 10  LotConfig      2919 non-null   object 
 11  LandSlope      2919 non-null   object 
 12  Neighborhood   2919 non-null   object 
 13  Condition1     2919 non-null   object 
 14  Condition2     2919 non-null   object 
 15  BldgType       2919 non-null   object 
 16  HouseStyle     2919 non-null   object 
 17  OverallQual    2919 non-null   int64  
 18  OverallC

In [6]:

houses.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,2919.000000,2919.000000,2433.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2896.000000,2918.000000,...,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,1460.000000
mean,1460.000000,57.137718,69.305795,10168.114080,6.089072,5.564577,1971.312778,1984.264474,102.201312,441.423235,...,93.709832,47.486811,23.098321,2.602261,16.062350,2.251799,50.825968,6.213087,2007.792737,180921.195890
std,842.787043,42.517628,23.344905,7886.996359,1.409947,1.113131,30.291442,20.894344,179.334253,455.610826,...,126.526589,67.575493,64.244246,25.188169,56.184365,35.663946,567.402211,2.714762,1.314964,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,730.500000,20.000000,59.000000,7478.000000,5.000000,5.000000,1953.500000,1965.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129975.000000
50%,1460.000000,50.000000,68.000000,9453.000000,6.000000,5.000000,1973.000000,1993.000000,0.000000,368.500000,...,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,2189.500000,70.000000,80.000000,11570.000000,7.000000,6.000000,2001.000000,2004.000000,164.000000,733.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,2919.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,1424.000000,742.000000,1012.000000,508.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000,755000.000000


In [9]:
pd.options.display.max_columns = 100
houses.describe(include="all")

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
count,2919.000000,2919.000000,2915,2433.000000,2919.000000,2919,198,2919,2919,2917,2919,2919,2919,2919,2919,2919,2919,2919.000000,2919.000000,2919.000000,2919.000000,2919,2919,2918,2918,1153,2896.000000,2919,2919,2919,2838,2837,2837,2840,2918.000000,2839,2918.000000,2918.000000,2918.000000,2919,2919,2919,2918,2919.000000,2919.000000,2919.000000,2919.000000,2917.000000,2917.000000,2919.000000,2919.000000,2919.000000,2919.000000,2918,2919.000000,2917,2919.000000,1499,2762,2760.000000,2760,2918.000000,2918.000000,2760,2760,2919,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,10,571,105,2919.000000,2919.000000,2919.000000,2918,2919,1460.000000
unique,NaN,NaN,5,NaN,NaN,2,2,4,4,2,5,3,25,9,8,5,8,NaN,NaN,NaN,NaN,6,8,15,16,3,NaN,4,5,6,4,4,4,6,NaN,6,NaN,NaN,NaN,6,5,2,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,7,NaN,5,6,NaN,3,NaN,NaN,5,5,3,NaN,NaN,NaN,NaN,NaN,NaN,3,4,4,NaN,NaN,NaN,9,6,NaN
top,NaN,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,NaN,NaN,NaN,NaN,Gable,CompShg,VinylSd,VinylSd,BrkFace,NaN,TA,TA,PConc,TA,TA,No,Unf,NaN,Unf,NaN,NaN,NaN,GasA,Ex,Y,SBrkr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TA,NaN,Typ,NaN,Gd,Attchd,NaN,Unf,NaN,NaN,TA,TA,Y,NaN,NaN,NaN,NaN,NaN,NaN,Ex,MnPrv,Shed,NaN,NaN,NaN,WD,Normal,NaN
freq,NaN,NaN,2265,NaN,NaN,2907,120,1859,2622,2916,2133,2778,443,2511,2889,2425,1471,NaN,NaN,NaN,NaN,2310,2876,1025,1014,879,NaN,1798,2538,1308,1283,2606,1904,851,NaN,2493,NaN,NaN,NaN,2874,1493,2723,2671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1492,NaN,2717,NaN,744,1723,NaN,1230,NaN,NaN,2604,2654,2641,NaN,NaN,NaN,NaN,NaN,NaN,4,329,95,NaN,NaN,NaN,2525,2402,NaN
mean,1460.000000,57.137718,NaN,69.305795,10168.114080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.089072,5.564577,1971.312778,1984.264474,NaN,NaN,NaN,NaN,NaN,102.201312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,441.423235,NaN,49.582248,560.772104,1051.777587,NaN,NaN,NaN,NaN,1159.581706,336.483727,4.694416,1500.759849,0.429894,0.061364,1.568003,0.380267,2.860226,1.044536,NaN,6.451524,NaN,0.597122,NaN,NaN,1978.113406,NaN,1.766621,472.874572,NaN,NaN,NaN,93.709832,47.486811,23.098321,2.602261,16.062350,2.251799,NaN,NaN,NaN,50.825968,6.213087,2007.792737,NaN,NaN,180921.195890
std,842.787043,42.517628,NaN,23.344905,7886.996359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.409947,1.113131,30.291442,20.894344,NaN,NaN,NaN,NaN,NaN,179.334253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455.610826,NaN,169.205611,439.543659,440.766258,NaN,NaN,NaN,NaN,392.362079,428.701456,46.396825,506.051045,0.524736,0.245687,0.552969,0.502872,0.822693,0.214462,NaN,1.569379,NaN,0.646129,NaN,NaN,25.574285,NaN,0.761624,215.394815,NaN,NaN,NaN,126.526589,67.575493,64.244246,25.188169,56.184365,35.663946,NaN,NaN,NaN,567.402211,2.714762,1.314964,NaN,NaN,79442.502883
min,1.000000,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1872.000000,1950.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,2.000000,NaN,0.000000,Na

In [10]:
houses.drop(columns=["Id"], inplace=True)

In [16]:
c_float = [x for x in range(len(houses.dtypes)) if houses.dtypes[x]==float]

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_26856\1249653163.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  c_float = [x for x in range(len(houses.dtypes)) if houses.dtypes[x]==float]
C:\Users\Jonatan\AppData\Local\Temp\ipykernel_26856\1249653163.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  c_float = [x for x in range(len(houses.dtypes)) if houses.dtypes[x]==float]
C:\Users\Jonatan\AppData\Local\Temp\ipykernel_26856\1249653163.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame beh

In [20]:
tipos = pd.DataFrame(houses.dtypes)

In [22]:
tipos[tipos[0]==float]

,0
LotFrontage,float64
MasVnrArea,float64
BsmtFinSF1,float64
BsmtFinSF2,float64
BsmtUnfSF,float64
TotalBsmtSF,float64
BsmtFullBath,float64
BsmtHalfBath,float64
GarageYrBlt,float64
GarageCars,float64


In [25]:
cols = tipos[tipos[0]==float].reset_index()["index"]

In [26]:
cols

0      LotFrontage
1       MasVnrArea
2       BsmtFinSF1
3       BsmtFinSF2
4        BsmtUnfSF
5      TotalBsmtSF
6     BsmtFullBath
7     BsmtHalfBath
8      GarageYrBlt
9       GarageCars
10      GarageArea
11       SalePrice
Name: index, dtype: object

In [27]:
dt = houses[cols]

In [29]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   LotFrontage   2433 non-null   float64
 1   MasVnrArea    2896 non-null   float64
 2   BsmtFinSF1    2918 non-null   float64
 3   BsmtFinSF2    2918 non-null   float64
 4   BsmtUnfSF     2918 non-null   float64
 5   TotalBsmtSF   2918 non-null   float64
 6   BsmtFullBath  2917 non-null   float64
 7   BsmtHalfBath  2917 non-null   float64
 8   GarageYrBlt   2760 non-null   float64
 9   GarageCars    2918 non-null   float64
 10  GarageArea    2918 non-null   float64
 11  SalePrice     1460 non-null   float64
dtypes: float64(12)
memory usage: 273.8 KB


In [30]:
dt = dt.dropna()

In [31]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1459
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   LotFrontage   1121 non-null   float64
 1   MasVnrArea    1121 non-null   float64
 2   BsmtFinSF1    1121 non-null   float64
 3   BsmtFinSF2    1121 non-null   float64
 4   BsmtUnfSF     1121 non-null   float64
 5   TotalBsmtSF   1121 non-null   float64
 6   BsmtFullBath  1121 non-null   float64
 7   BsmtHalfBath  1121 non-null   float64
 8   GarageYrBlt   1121 non-null   float64
 9   GarageCars    1121 non-null   float64
 10  GarageArea    1121 non-null   float64
 11  SalePrice     1121 non-null   float64
dtypes: float64(12)
memory usage: 113.9 KB


In [34]:
# Esta vez, nuestro modelo serán el 80% de los datos y el test el 20% de los datos
X = dt.drop(columns=["SalePrice"])
X_train, X_test, y_train, y_test = train_test_split(X, dt["SalePrice"], test_size = 0.2, random_state = 42)

In [37]:
X_train

,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea
460,75.0,110.0,544.0,0.0,288.0,832.0,1.0,0.0,2009.0,2.0,552.0
479,50.0,435.0,168.0,0.0,739.0,907.0,0.0,0.0,1995.0,2.0,672.0
500,21.0,285.0,356.0,0.0,316.0,672.0,0.0,0.0,1973.0,1.0,264.0
683,90.0,215.0,1059.0,0.0,567.0,1626.0,1.0,0.0,2002.0,3.0,702.0
596,60.0,0.0,0.0,0.0,684.0,684.0,0.0,0.0,1930.0,1.0,216.0
...,...,...,...,...,...,...,...,...,...,...,...
598,80.0,459.0,1283.0,147.0,0.0,1430.0,1.0,0.0,1977.0,2.0,621.0
162,95.0,226.0,1201.0,0.0,340.0,1541.0,0.0,0.0,2005.0,2.0,532.0
1364,30.0,0.0,0.0,0.0,600.0,600.0,0.0,0.0,2005.0,2.0,480.0
1426,81.0,448.0,1000.0,0.0,223.0,1223.0,1.0,0.0,1994.0,2.0,525.0


In [ ]:
#from sklearn.model_selection import StratifiedShuffleSplit
#split = StratifiedShuffleSplit(n_splits=1, test_size=, random_state=)
#for train_index, test_index in split.split():
#    X_train = .iloc[train_index]
#    X_test = .iloc[test_index]

In [43]:
# Crea una nueva regresión lineal (no uses el mismo nombre que la anterior!!!) usando X_train e y_train
# Realiza la regresión lineal

# Creación del modelo utilizando matrices como en scikitlearn
# ==============================================================================
# A la matriz de predictores se le tiene que añadir una columna de 1s para el intercept del modelo

# La convenciones determinan que las features se llamen X 
# Además, tendremos habitualmente un conjunto de train y otro de test (con un split del 80/20, habitualmente)
# Por lo tanto, X_train contiene las features del train
# Este caso es tan sencillo y con pocos datos que no tenemos X_test, pero aparecerá
#X_train = USA_Housing[["Avg. Area Income",	"Avg. Area House Age",	"Avg. Area Number of Rooms", 	"Avg. Area Number of Bedrooms",	"Area Population"]]

# Añadimos una columna de 1 porque necesitamos una constante que determine el intercept o B0, el punto donde corta la recta con el eje Y
# Por eso añadimos una columna constante de 0s
# Veremos que estoy aparece en muchos otros modelos (en redes neuronales se llama bias)
X_train = sm.add_constant(X_train, prepend=True)
# OLS: creamos el modelo del tipo Ordinary Least Squares (por alguna razón creo que una vez dije Optimal pero es Ordinary)
modelo_ = sm.OLS(endog=y_train, exog=X_train)
# Ajustamos el modelo, es decir, hacemos el calculo de la mejor recta posible según los criterios del OLS
modelo_ = modelo_.fit()
print(modelo_.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.602
Model:                            OLS   Adj. R-squared:                  0.598
Method:                 Least Squares   F-statistic:                     134.1
Date:                Mon, 13 May 2024   Prob (F-statistic):          1.20e-169
Time:                        19:07:43   Log-Likelihood:                -10988.
No. Observations:                 896   AIC:                         2.200e+04
Df Residuals:                     885   BIC:                         2.205e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.033e+06    1.7e+05     -6.068   

In [46]:
dt.corr()

,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea,SalePrice
LotFrontage,1.000000,0.189969,0.241352,0.049305,0.115306,0.387620,0.118088,0.000434,0.069878,0.286587,0.356851,0.344270
MasVnrArea,0.189969,1.000000,0.285331,-0.075261,0.110067,0.384434,0.110379,-0.007035,0.277095,0.375269,0.382162,0.488658
BsmtFinSF1,0.241352,0.285331,1.000000,-0.035780,-0.502225,0.530917,0.651727,0.061963,0.160356,0.196443,0.286657,0.390301
BsmtFinSF2,0.049305,-0.075261,-0.035780,1.000000,-0.220190,0.094079,0.168559,0.059148,-0.075477,-0.075477,-0.047959,-0.028021
BsmtUnfSF,0.115306,0.110067,-0.502225,-0.220190,1.000000,0.404510,-0.431243,-0.102936,0.208915,0.277064,0.235329,0.213129
TotalBsmtSF,0.387620,0.384434,0.530917,0.094079,0.404510,1.000000,0.308963,-0.017929,0.352877,0.459657,0.522051,0.615612
BsmtFullBath,0.118088,0.110379,0.651727,0.168559,-0.431243,0.308963,1.000000,-0.139866,0.125844,0.133961,0.189143,0.236737
BsmtHalfBath,0.000434,-0.007035,0.061963,0.059148,-0.102936,-0.017929,-0.139866,1.000000,-0.073415,-0.049748,-0.051098,-0.036513
GarageYrBlt,0.069878,0.277095,0.160356,-0.075477,0.208915,0.352877,0.125844,-0.073415,1.000000,0.600903,0.592635,0.504753
GarageCars,0.286587,0.375269,0.196443,-0.075477,0.277064,0.459657,0.133961,-0.049748,0.600903,1.000000,0.839415,0.647034


In [47]:
modelo_.resid.mean()

-4.357129260565021e-10

In [51]:
X_test = sm.add_constant(X_test, prepend=True)

In [53]:
X_test

,const,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea
1297,1.0,35.0,82.0,547.0,0.0,0.0,547.0,1.0,0.0,2005.0,2.0,525.0
132,1.0,75.0,0.0,405.0,0.0,658.0,1063.0,1.0,0.0,1974.0,2.0,624.0
300,1.0,90.0,56.0,841.0,0.0,324.0,1165.0,1.0,0.0,1953.0,1.0,375.0
1307,1.0,60.0,0.0,746.0,0.0,244.0,990.0,1.0,0.0,2000.0,2.0,480.0
778,1.0,60.0,320.0,0.0,0.0,0.0,0.0,0.0,0.0,1977.0,2.0,630.0
...,...,...,...,...,...,...,...,...,...,...,...,...
401,1.0,65.0,0.0,24.0,0.0,1286.0,1310.0,0.0,0.0,2005.0,2.0,400.0
543,1.0,34.0,182.0,584.0,139.0,0.0,723.0,1.0,0.0,1998.0,1.0,367.0
1062,1.0,85.0,0.0,0.0,0.0,662.0,662.0,0.0,0.0,1945.0,2.0,560.0
1026,1.0,73.0,324.0,697.0,0.0,571.0,1268.0,1.0,0.0,1960.0,2.0,461.0


In [54]:
preds = modelo_.predict(exog=X_test)

In [55]:
mae = abs(y_test - preds).mean()

In [56]:
mae

37675.087724072815

In [58]:
mae/y_test.mean()

0.1940673876617896

In [64]:
sum(y_test - preds)

728381.645078709

In [65]:
X_train.describe()

,const,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea
count,896.0,896.000000,896.000000,896.000000,896.000000,896.000000,896.000000,896.000000,896.000000,896.000000,896.000000,896.000000
mean,1.0,70.833705,105.232143,439.571429,46.202009,587.997768,1073.771205,0.402902,0.053571,1977.658482,1.860491,498.852679
std,0.0,24.374596,184.822457,473.242810,163.298597,444.989654,455.598741,0.508643,0.230202,25.839243,0.661437,195.985653
min,1.0,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1900.000000,1.000000,160.000000
25%,1.0,60.000000,0.000000,0.000000,0.000000,254.000000,802.750000,0.000000,0.000000,1959.000000,1.000000,352.000000
50%,1.0,70.000000,0.000000,370.000000,0.000000,504.000000,992.500000,0.000000,0.000000,1979.000000,2.000000,480.000000
75%,1.0,80.000000,168.000000,700.500000,0.000000,816.500000,1314.500000,1.000000,0.000000,2003.000000,2.000000,583.750000
max,1.0,313.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,2.000000,2.000000,2010.000000,4.000000,1418.000000


In [67]:
X_test.describe()

,const,LotFrontage,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath,GarageYrBlt,GarageCars,GarageArea
count,225.0,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000,225.000000
mean,1.0,69.995556,121.453333,433.733333,38.168889,618.528889,1090.431111,0.457778,0.062222,1981.351111,1.951111,519.293333
std,0.0,23.874579,207.019234,449.511004,133.981502,470.894087,404.316151,0.525463,0.242097,25.601972,0.621283,170.481758
min,1.0,21.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1910.000000,1.000000,180.000000
25%,1.0,57.000000,0.000000,0.000000,0.000000,240.000000,806.000000,0.000000,0.000000,1966.000000,2.000000,400.000000
50%,1.0,69.000000,0.000000,383.000000,0.000000,528.000000,1075.000000,0.000000,0.000000,1990.000000,2.000000,528.000000
75%,1.0,82.000000,183.000000,705.000000,0.000000,939.000000,1346.000000,1.000000,0.000000,2004.000000,2.000000,626.000000
max,1.0,182.000000,1170.000000,2188.000000,872.000000,1752.000000,2524.000000,2.000000,1.000000,2010.000000,4.000000,936.000000


In [68]:
y_test.mean()

194134.04888888888

In [69]:
y_train.mean()

183339.54910714287

In [71]:
houses.describe(include="all")

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
count,2919.000000,2915,2433.000000,2919.000000,2919,198,2919,2919,2917,2919,2919,2919,2919,2919,2919,2919,2919.000000,2919.000000,2919.000000,2919.000000,2919,2919,2918,2918,1153,2896.000000,2919,2919,2919,2838,2837,2837,2840,2918.000000,2839,2918.000000,2918.000000,2918.000000,2919,2919,2919,2918,2919.000000,2919.000000,2919.000000,2919.000000,2917.000000,2917.000000,2919.000000,2919.000000,2919.000000,2919.000000,2918,2919.000000,2917,2919.000000,1499,2762,2760.000000,2760,2918.000000,2918.000000,2760,2760,2919,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,10,571,105,2919.000000,2919.000000,2919.000000,2918,2919,1460.000000
unique,NaN,5,NaN,NaN,2,2,4,4,2,5,3,25,9,8,5,8,NaN,NaN,NaN,NaN,6,8,15,16,3,NaN,4,5,6,4,4,4,6,NaN,6,NaN,NaN,NaN,6,5,2,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,7,NaN,5,6,NaN,3,NaN,NaN,5,5,3,NaN,NaN,NaN,NaN,NaN,NaN,3,4,4,NaN,NaN,NaN,9,6,NaN
top,NaN,RL,NaN,NaN,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,NaN,NaN,NaN,NaN,Gable,CompShg,VinylSd,VinylSd,BrkFace,NaN,TA,TA,PConc,TA,TA,No,Unf,NaN,Unf,NaN,NaN,NaN,GasA,Ex,Y,SBrkr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TA,NaN,Typ,NaN,Gd,Attchd,NaN,Unf,NaN,NaN,TA,TA,Y,NaN,NaN,NaN,NaN,NaN,NaN,Ex,MnPrv,Shed,NaN,NaN,NaN,WD,Normal,NaN
freq,NaN,2265,NaN,NaN,2907,120,1859,2622,2916,2133,2778,443,2511,2889,2425,1471,NaN,NaN,NaN,NaN,2310,2876,1025,1014,879,NaN,1798,2538,1308,1283,2606,1904,851,NaN,2493,NaN,NaN,NaN,2874,1493,2723,2671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1492,NaN,2717,NaN,744,1723,NaN,1230,NaN,NaN,2604,2654,2641,NaN,NaN,NaN,NaN,NaN,NaN,4,329,95,NaN,NaN,NaN,2525,2402,NaN
mean,57.137718,NaN,69.305795,10168.114080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.089072,5.564577,1971.312778,1984.264474,NaN,NaN,NaN,NaN,NaN,102.201312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,441.423235,NaN,49.582248,560.772104,1051.777587,NaN,NaN,NaN,NaN,1159.581706,336.483727,4.694416,1500.759849,0.429894,0.061364,1.568003,0.380267,2.860226,1.044536,NaN,6.451524,NaN,0.597122,NaN,NaN,1978.113406,NaN,1.766621,472.874572,NaN,NaN,NaN,93.709832,47.486811,23.098321,2.602261,16.062350,2.251799,NaN,NaN,NaN,50.825968,6.213087,2007.792737,NaN,NaN,180921.195890
std,42.517628,NaN,23.344905,7886.996359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.409947,1.113131,30.291442,20.894344,NaN,NaN,NaN,NaN,NaN,179.334253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455.610826,NaN,169.205611,439.543659,440.766258,NaN,NaN,NaN,NaN,392.362079,428.701456,46.396825,506.051045,0.524736,0.245687,0.552969,0.502872,0.822693,0.214462,NaN,1.569379,NaN,0.646129,NaN,NaN,25.574285,NaN,0.761624,215.394815,NaN,NaN,NaN,126.526589,67.575493,64.244246,25.188169,56.184365,35.663946,NaN,NaN,NaN,567.402211,2.714762,1.314964,NaN,NaN,79442.502883
min,20.000000,NaN,21.000000,1300.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1872.000000,1950.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,2.000000,NaN,0.000000,NaN,NaN,1895.000000,NaN,0.000000,0.000000,NaN,NaN,NaN,0.00000

In [72]:
houses["Street"].unique()

array(['Pave', 'Grvl'], dtype=object)

In [75]:
houses["Street"] = houses["Street"].map({"Pave": 0, "Grvl": 1})

In [77]:
houses["CentralAir"].unique()

array(['Y', 'N'], dtype=object)

In [78]:
houses["CentralAir"] = houses["CentralAir"].map({"N": 0, "Y": 1})

In [84]:
cols = list(cols.values) + ["Street", "CentralAir"]
dt = houses[cols]

In [89]:
dt = dt.dropna()

In [90]:
# Esta vez, nuestro modelo serán el 80% de los datos y el test el 20% de los datos
dt = dt.dropna()
X = dt.drop(columns=["SalePrice"])
X_train, X_test, y_train, y_test = train_test_split(X, dt["SalePrice"], test_size = 0.2, random_state = 42)

In [91]:
# Crea una nueva regresión lineal (no uses el mismo nombre que la anterior!!!) usando X_train e y_train
# Realiza la regresión lineal

# Creación del modelo utilizando matrices como en scikitlearn
# ==============================================================================
# A la matriz de predictores se le tiene que añadir una columna de 1s para el intercept del modelo

# La convenciones determinan que las features se llamen X 
# Además, tendremos habitualmente un conjunto de train y otro de test (con un split del 80/20, habitualmente)
# Por lo tanto, X_train contiene las features del train
# Este caso es tan sencillo y con pocos datos que no tenemos X_test, pero aparecerá
#X_train = USA_Housing[["Avg. Area Income",	"Avg. Area House Age",	"Avg. Area Number of Rooms", 	"Avg. Area Number of Bedrooms",	"Area Population"]]

# Añadimos una columna de 1 porque necesitamos una constante que determine el intercept o B0, el punto donde corta la recta con el eje Y
# Por eso añadimos una columna constante de 0s
# Veremos que estoy aparece en muchos otros modelos (en redes neuronales se llama bias)
X_train = sm.add_constant(X_train, prepend=True)
# OLS: creamos el modelo del tipo Ordinary Least Squares (por alguna razón creo que una vez dije Optimal pero es Ordinary)
modelo_ = sm.OLS(endog=y_train, exog=X_train)
# Ajustamos el modelo, es decir, hacemos el calculo de la mejor recta posible según los criterios del OLS
modelo_ = modelo_.fit()
print(modelo_.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.610
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     115.3
Date:                Mon, 13 May 2024   Prob (F-statistic):          2.04e-171
Time:                        20:20:32   Log-Likelihood:                -10979.
No. Observations:                 896   AIC:                         2.198e+04
Df Residuals:                     883   BIC:                         2.205e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -8.729e+05   1.76e+05     -4.971   

In [92]:
houses.describe(include="all")

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
count,2919.000000,2915,2433.000000,2919.000000,2919.000000,198,2919,2919,2917,2919,2919,2919,2919,2919,2919,2919,2919.000000,2919.000000,2919.000000,2919.000000,2919,2919,2918,2918,1153,2896.000000,2919,2919,2919,2838,2837,2837,2840,2918.000000,2839,2918.000000,2918.000000,2918.000000,2919,2919,2919.000000,2918,2919.000000,2919.000000,2919.000000,2919.000000,2917.000000,2917.000000,2919.000000,2919.000000,2919.000000,2919.000000,2918,2919.000000,2917,2919.000000,1499,2762,2760.000000,2760,2918.000000,2918.000000,2760,2760,2919,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,2919.000000,10,571,105,2919.000000,2919.000000,2919.000000,2918,2919,1460.000000
unique,NaN,5,NaN,NaN,NaN,2,4,4,2,5,3,25,9,8,5,8,NaN,NaN,NaN,NaN,6,8,15,16,3,NaN,4,5,6,4,4,4,6,NaN,6,NaN,NaN,NaN,6,5,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,7,NaN,5,6,NaN,3,NaN,NaN,5,5,3,NaN,NaN,NaN,NaN,NaN,NaN,3,4,4,NaN,NaN,NaN,9,6,NaN
top,NaN,RL,NaN,NaN,NaN,Grvl,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,NaN,NaN,NaN,NaN,Gable,CompShg,VinylSd,VinylSd,BrkFace,NaN,TA,TA,PConc,TA,TA,No,Unf,NaN,Unf,NaN,NaN,NaN,GasA,Ex,NaN,SBrkr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TA,NaN,Typ,NaN,Gd,Attchd,NaN,Unf,NaN,NaN,TA,TA,Y,NaN,NaN,NaN,NaN,NaN,NaN,Ex,MnPrv,Shed,NaN,NaN,NaN,WD,Normal,NaN
freq,NaN,2265,NaN,NaN,NaN,120,1859,2622,2916,2133,2778,443,2511,2889,2425,1471,NaN,NaN,NaN,NaN,2310,2876,1025,1014,879,NaN,1798,2538,1308,1283,2606,1904,851,NaN,2493,NaN,NaN,NaN,2874,1493,NaN,2671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1492,NaN,2717,NaN,744,1723,NaN,1230,NaN,NaN,2604,2654,2641,NaN,NaN,NaN,NaN,NaN,NaN,4,329,95,NaN,NaN,NaN,2525,2402,NaN
mean,57.137718,NaN,69.305795,10168.114080,0.004111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.089072,5.564577,1971.312778,1984.264474,NaN,NaN,NaN,NaN,NaN,102.201312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,441.423235,NaN,49.582248,560.772104,1051.777587,NaN,NaN,0.932854,NaN,1159.581706,336.483727,4.694416,1500.759849,0.429894,0.061364,1.568003,0.380267,2.860226,1.044536,NaN,6.451524,NaN,0.597122,NaN,NaN,1978.113406,NaN,1.766621,472.874572,NaN,NaN,NaN,93.709832,47.486811,23.098321,2.602261,16.062350,2.251799,NaN,NaN,NaN,50.825968,6.213087,2007.792737,NaN,NaN,180921.195890
std,42.517628,NaN,23.344905,7886.996359,0.063996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.409947,1.113131,30.291442,20.894344,NaN,NaN,NaN,NaN,NaN,179.334253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,455.610826,NaN,169.205611,439.543659,440.766258,NaN,NaN,0.250318,NaN,392.362079,428.701456,46.396825,506.051045,0.524736,0.245687,0.552969,0.502872,0.822693,0.214462,NaN,1.569379,NaN,0.646129,NaN,NaN,25.574285,NaN,0.761624,215.394815,NaN,NaN,NaN,126.526589,67.575493,64.244246,25.188169,56.184365,35.663946,NaN,NaN,NaN,567.402211,2.714762,1.314964,NaN,NaN,79442.502883
min,20.000000,NaN,21.000000,1300.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,1.000000,1872.000000,1950.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,2.000000,NaN,0.000000,NaN,NaN,1895.0

In [98]:
dum = pd.get_dummies(houses["LotShape"], dtype=int)

In [100]:
dt = pd.merge(dt, dum, right_index=True, left_index=True)

In [102]:
dt.drop(columns=["Reg"], inplace=True)

In [127]:
# Esta vez, nuestro modelo serán el 80% de los datos y el test el 20% de los datos
dt = dt.dropna()
X = dt.drop(columns=["SalePrice"])
X_train, X_test, y_train, y_test = train_test_split(X, dt["SalePrice"], test_size = 0.2, random_state = 42)

In [128]:
# Crea una nueva regresión lineal (no uses el mismo nombre que la anterior!!!) usando X_train e y_train
# Realiza la regresión lineal

# Creación del modelo utilizando matrices como en scikitlearn
# ==============================================================================
# A la matriz de predictores se le tiene que añadir una columna de 1s para el intercept del modelo

# La convenciones determinan que las features se llamen X 
# Además, tendremos habitualmente un conjunto de train y otro de test (con un split del 80/20, habitualmente)
# Por lo tanto, X_train contiene las features del train
# Este caso es tan sencillo y con pocos datos que no tenemos X_test, pero aparecerá
#X_train = USA_Housing[["Avg. Area Income",	"Avg. Area House Age",	"Avg. Area Number of Rooms", 	"Avg. Area Number of Bedrooms",	"Area Population"]]

# Añadimos una columna de 1 porque necesitamos una constante que determine el intercept o B0, el punto donde corta la recta con el eje Y
# Por eso añadimos una columna constante de 0s
# Veremos que estoy aparece en muchos otros modelos (en redes neuronales se llama bias)
X_train = sm.add_constant(X_train, prepend=True)
# OLS: creamos el modelo del tipo Ordinary Least Squares (por alguna razón creo que una vez dije Optimal pero es Ordinary)
modelo_ = sm.OLS(endog=y_train, exog=X_train)
# Ajustamos el modelo, es decir, hacemos el calculo de la mejor recta posible según los criterios del OLS
modelo_ = modelo_.fit()
print(modelo_.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.627
Model:                            OLS   Adj. R-squared:                  0.621
Method:                 Least Squares   F-statistic:                     98.73
Date:                Mon, 13 May 2024   Prob (F-statistic):          1.09e-176
Time:                        21:00:07   Log-Likelihood:                -10959.
No. Observations:                 896   AIC:                         2.195e+04
Df Residuals:                     880   BIC:                         2.203e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -7.655e+05   1.75e+05     -4.380   

In [105]:
X_test = sm.add_constant(X_test, prepend=True)
preds = modelo_.predict(X_test)
mae = abs(y_test -preds).mean()

In [108]:
(y_test -preds).sum()

309519.4148733274

In [106]:
mae

37817.13332458655

In [107]:
mae/y_test.mean()

0.19479907590157405

In [ ]:
model = LinearRegression()
model.fit(X, y)

In [150]:
model_r = Ridge(alpha=1000,fit_intercept=False, random_state=42, solver="cholesky", max_iter=1000)
model_r.fit(X_train, y_train)

Ridge(alpha=1000, fit_intercept=False, max_iter=1000, random_state=42,
      solver='cholesky')

In [151]:
model_r.coef_

array([-126.42182977,  225.71434767,   86.80335053,   19.31243343,
          5.88697579,   12.52841387,   37.72782308, 1164.19933607,
       -324.62985421,   12.44667817, 5196.69730562,  129.89520867,
       -437.66334808, 1477.81666348, 3394.4936582 ,  899.79539132,
       -257.615152  ])

In [163]:
model_l = Lasso(alpha=1000,fit_intercept=False, random_state=42)
model_l.fit(X_train, y_train)

c:\Users\Jonatan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.228e+11, tolerance: 3.604e+09
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=1000, fit_intercept=False, random_state=42)

In [164]:
model_l.coef_

array([-0.00000000e+00,  2.39849521e+02,  7.79958465e+01,  5.92540276e+01,
        4.69096389e+01,  4.81356900e+01, -3.62360044e+00,  1.19604461e+03,
       -0.00000000e+00, -4.71821574e-01,  4.04351506e+04,  3.33245399e+01,
       -0.00000000e+00,  1.09464506e+04,  1.45742186e+04,  0.00000000e+00,
       -0.00000000e+00])

In [175]:
model_e = ElasticNet(alpha=1, l1_ratio=0.5, fit_intercept=False)
model_e.fit(X_train, y_train)

c:\Users\Jonatan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.307e+12, tolerance: 3.604e+09
  model = cd_fast.enet_coordinate_descent(


ElasticNet(alpha=1, fit_intercept=False)

In [176]:
model_e.coef_

array([-2.65655233e+02,  2.28144549e+02,  8.55225856e+01,  6.35561421e+01,
        5.04037665e+01,  5.69298378e+01, -7.61183495e+00,  2.20345388e+03,
       -6.62305188e+02,  1.05494083e+01,  1.01675320e+04,  1.15820832e+02,
       -9.52295054e+02,  3.06888088e+03,  6.29776135e+03,  1.97069309e+03,
       -5.30687399e+02])